In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
from fiona.crs import from_epsg

In [2]:
import urllib, json, requests 
import geojson

In [3]:
from ipyleaflet import Map, GeoData, GeoJSON, basemaps, basemap_to_tiles, Icon, Circle, Marker, LayerGroup, WidgetControl
import ipywidgets as widgets
from ipywidgets import Button 
from IPython.display import display, clear_output

In [4]:
#need this to stop numpy from returning truncated arrays 
import sys
np.set_printoptions(threshold=sys.maxsize)

In [5]:
#drawing basic map
center = (40.7210907,-73.9877836)
basemap = basemap_to_tiles(basemaps.CartoDB.Positron)

m = Map(layers=(basemap, ), center=center, zoom=15, min_zoom = 7, max_zoom = 20)

In [6]:
def extract_location():       
    global gdf, lat, lon
    
    lat = str(markerlocation[0])
    lon = str(markerlocation[1])
    
    df2 = pd.DataFrame(markerlocation)
    df=df2.transpose()
    df.columns=['Latitude','Longitude']

    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs='epsg:4326')
    
    return gdf

In [7]:
draggable=False
marker_opacity=1
icon = Icon(icon_url='icon.png', icon_size=[15, 15])

marker = Marker(location=center, draggable=draggable, icon=icon, opacity=marker_opacity)

markerlocation = marker.location 

layer_group = LayerGroup(layers=(marker, ))
m.add_layer(layer_group)
   
def update_marker(**kwargs):
    
    if kwargs.get('type') == 'click':
        layer_group.clear_layers();
        
        marker = Marker(location=kwargs.get('coordinates'), draggable=draggable, icon=icon, opacity=marker_opacity, options=['rise_on_hover'])  
        
        global markerlocation
        markerlocation = marker.location 
        
        layer_group.add_layer(marker)
    
        draw_update_buffer(**kwargs)
    
m.on_interaction(update_marker)

In [8]:
def draw_update_buffer(**kwargs):     
    m.on_interaction(update_marker)
    extract_location()
    
    global half_mi
    half_mi=gdf.copy()
    half_mi['geometry'] = half_mi.geometry.buffer(.004,  cap_style=1, join_style=1)

    map_extent = gdf.copy()
    map_extent['geometry'] = map_extent.buffer(1,  cap_style=1, join_style=1)

    diff = gpd.overlay(map_extent, half_mi, how='difference')
    
    half_mi_difference = GeoData(geo_dataframe = diff,
                       style={'color': "black", \
                              'fillColor': "#000000", \
                              'fillOpacity': .2, \
                              'opacity': 1, \
                              'weight': 2},
                       name = "Test", crs='epsg:4326')

    layer_group.add_layer(half_mi_difference) 

In [9]:
# m

In [52]:
def import_censustracts():
    draw_update_buffer()
    
    bounding_box = half_mi.envelope
    df = gpd.GeoDataFrame(gpd.GeoSeries(bounding_box), columns=['geometry'])
    minx, miny, maxx, maxy = df.geometry.total_bounds
    bounds = minx, miny, maxx, maxy

    # census tracts link
    endpoint = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/4/query'
    s = requests.session()
    s.params = {
        'geometry': str(bounds),
        'geometryType': 'esriGeometryEnvelope',
        'inSR': 4326,
        'spatialRel': 'esriSpatialRelIntersects',
        'outFields': 'GEOID,STATE,COUNTY,TRACT,NAME,STGEOMETRY,OBJECTID',
        'returnGeometry': True,
        'f': 'geojson',        
    }
    start = 0
    done = False
    features = []
    crs = None
    while not done:
        r = s.get(endpoint, params={
            'resultOffset': start,
            'resultRecordCount': 32,
        })
        censusgeo = geojson.loads(r.text)
        newfeats = censusgeo.__geo_interface__['features']
        if newfeats:
            features.extend(newfeats)
            crs=censusgeo.__geo_interface__['crs']
            start += len(newfeats)
            print("Received", len(newfeats), "entries,", start, "total")
        else:
            done = True
    
    global tracts
    tracts = gpd.GeoDataFrame.from_features(features, crs=crs)
    return tracts

import_censustracts()

Received 12 entries, 12 total
Received 1 entries, 13 total


,geometry,GEOID,STATE,COUNTY,TRACT,NAME,OBJECTID
0,"POLYGON ((-73.98986 40.72053, -73.98962 40.720...",36061003001,36,061,003001,Census Tract 30.01,10843
1,"POLYGON ((-73.99326 40.72235, -73.99352 40.721...",36061003601,36,061,003601,Census Tract 36.01,10846
2,"POLYGON ((-73.99155 40.72709, -73.99179 40.726...",36061003800,36,061,003800,Census Tract 38,10847
3,"POLYGON ((-73.98788 40.71741, -73.98837 40.716...",36061001402,36,061,001402,Census Tract 14.02,26519
4,"POLYGON ((-73.98845 40.72328, -73.98864 40.722...",36061003002,36,061,003002,Census Tract 30.02,40986
5,"POLYGON ((-73.98454 40.71639, -73.98501 40.715...",36061001200,36,061,001200,Census Tract 12,45321
6,"POLYGON ((-73.99233 40.72491, -73.99260 40.724...",36061003602,36,061,003602,Census Tract 36.02,52962
7,"POLYGON ((-73.98705 40.72520, -73.98750 40.724...",36061003200,36,061,003200,Census Tract 32,56775
8,"POLYGON ((-73.99750 40.71407, -73.99744 40.714...",36061001600,36,061,001600,Census Tract 16,71680
9,"POLYGON ((-73.99442 40.71939, -73.99481 40.718...",36061001800,36,061,001800,Census Tract 18,71681


In [53]:
def download_acs():  
    state = tracts["STATE"].unique().tolist()
    state = ', '.join(map(str, state)).replace(" ", "")

    tract = tracts["TRACT"].unique().tolist()
    tract = ', '.join(map(str, tract)).replace(" ", "") 

    county = tracts["COUNTY"].unique().tolist()
    county = ', '.join(map(str, county)).replace(" ", "") 

    api_key = '9330dc4bf086a84f19fb412bb15f232507301de6'
    acs_url = f'https://api.census.gov/data/2018/acs/acs5/subject/'
    
    global acs_variables
    acs_variables_initial = 'S1603_C02_002E,S1603_C02_003E,S1603_C02_004E,S1603_C04_002E,S1603_C04_003E,S1603_C04_004E,S1601_C01_005E,S1601_C01_006E,S1601_C01_007E,S1601_C01_009E,S1601_C01_010E,S1601_C01_011E,S1601_C01_013E,S1601_C01_014E,S1601_C01_015E,S1601_C01_017E,S1601_C01_018E,S1601_C01_019E,S1901_C01_002E,S1901_C01_003E,S1901_C01_004E,S1901_C01_005E,S1901_C01_006E,S1901_C01_007E,S1901_C01_008E,S1901_C01_009E,S1901_C01_010E,S1901_C01_011E,S1901_C04_002E,S1901_C04_003E,S1901_C04_004E,S1901_C04_005E,S1901_C04_006E,S1901_C04_007E,S1901_C04_008E,S1901_C04_009E,S1901_C04_010E,S1901_C04_011E'
    acs_variables_additional = 'S1501_C01_002E,S1501_C01_004E,S1501_C01_003E,S1501_C01_005E,S1501_C01_017E,S1501_C01_018E,S1501_C01_020E,S1501_C01_021E,S1501_C01_023E,S1501_C01_024E,S1501_C01_025E,S1501_C01_026E,S1501_C03_002E,S1501_C03_003E,S1501_C03_004E,S1501_C03_005E,S1501_C03_017E,S1501_C03_018E,S1501_C03_020E,S1501_C03_021E,S1501_C03_023E,S1501_C03_024E,S1501_C03_026E,S1501_C03_027E,S1501_C05_002E,S1501_C05_003E,S1501_C05_004E,S1501_C05_005E,S1501_C05_017E,S1501_C05_018E,S1501_C05_020E,S1501_C05_021E,S1501_C05_023E,S1501_C05_024E,S1501_C05_026E,S1501_C05_027E,S1401_C01_030E,S1401_C01_032E,S1401_C01_034E,S1101_C01_003E,S1101_C05_001E'
    acs_variables = acs_variables_initial + "," + acs_variables_additional
    
    get_acs_initial = f'{acs_url}?&get={acs_variables_initial}&for=tract:{tract}&in=state:{state}%20county:{county}&key={api_key}'
    get_acs_additional = f'{acs_url}?&get={acs_variables_additional}&for=tract:{tract}&in=state:{state}%20county:{county}&key={api_key}'

    data_acs_initial=requests.get(get_acs_initial).json()
    data_acs_additional=requests.get(get_acs_additional).json()
    
    global acs
    acs_initial=pd.DataFrame(data_acs_initial[1:], columns=data_acs_initial[0])
    acs_additional=pd.DataFrame(data_acs_additional[1:], columns=data_acs_additional[0])

    acs=pd.merge(acs_initial, acs_additional, on='tract', how='left')

download_acs()

In [54]:
#any null rows?
# test = acs.columns[acs.isnull().any()]
# test

In [55]:
# acs = acs.drop(['S0901_C01_033E', 'S0901_C01_034E', 'S0902_C01_003E', 'S0902_C01_004E','S0902_C01_005E'], axis=1)

In [56]:
def clean_combine_census_and_geographic_data():
    global acs_site_sum, acs_site
    tracts["area"]=tracts.area
    acs_tracts = pd.merge(tracts, acs, left_on='TRACT', right_on='tract', how='left')
    
    acs_site = gpd.overlay(half_mi, acs_tracts, how='intersection')
    acs_site["area_clipped"]=acs_site.area 
    acs_site["ratio"] = acs_site["area_clipped"]/acs_site["area"]
    
    cols = acs_variables.split(",")
    acs_site[cols] = acs_site[cols].apply(pd.to_numeric, errors='coerce', axis=1)
    
    # if 'area_clipped' not in cols:
    #     cols.append("area_clipped")
    
    temp_df = acs_site[cols]    
    temp_df = temp_df.mul(acs_site.ratio, 0)
    acs_site.update(temp_df)

    acs_site_sum = pd.DataFrame(acs_site[cols].sum())

    acs_site_sum.reset_index(inplace=True)
    acs_site_sum.columns = ['variables', 'sum_in_area']
    
clean_combine_census_and_geographic_data()

In [57]:
data_dict = pd.read_csv("data-dictionary.csv")
data_dict

,sex,age_group,variable_group,variables,variable_name,ages
0,Male Female Both,5 to 17 years,Language Spoken At Home,S1603_C02_002E,Speak Only English at Home,"5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17"
1,Male Female Both,18 to 64 years,Language Spoken At Home,S1603_C02_003E,Speak Only English at Home,"18, 19, 20, 21, 22, 23, 24, 25,26, 27, 28, 29,..."
2,Male Female Both,65 years and over,Language Spoken At Home,S1603_C02_004E,Speak Only English at Home,"64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75..."
3,Male Female Both,5 to 17 years,Language Spoken At Home,S1601_C01_005E,Spanish,"5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17"
4,Male Female Both,18 to 64 years,Language Spoken At Home,S1601_C01_006E,Spanish,"18, 19, 20, 21, 22, 23, 24, 25,26, 27, 28, 29,..."
...,...,...,...,...,...,...
71,Both,18 to 24 years,School Enrollment,S1401_C01_030E,Enrolled in college or graduate school,"18, 19, 20, 21, 22, 23, 24"
72,Male,18 to 24 years,School Enrollment,S1401_C01_032E,Enrolled in college or graduate school,"18, 19, 20, 21, 22, 23, 24"
73,Female,18 to 24 years,School Enrollment,S1401_C01_034E,Enrolled in college or graduate school,"18, 19, 20, 21, 22, 23, 24"
74,Male Female Both,all ages,Households and Families,S1101_C01_003E,Family households,"5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,..."


In [58]:
# only use this if i decide to add in the option to select all
ALL = 'ALL'
def user_options_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique
# 

In [59]:
def user_options_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

In [60]:
output = widgets.Output()

In [61]:
data_output = widgets.Output()

In [62]:
def user_selection():
    
    output.clear_output()
    data_output.clear_output() 
    
    global selected_age, selected_gender, selected_percentile, text_generation_button, selection_filter, variable_inputs
#     selected_age = widgets.Dropdown(options = user_options_sorted_values(data_dict.age_group),\
#                                     value = "5 to 17 Years")
    selected_age = widgets.BoundedIntText(min=5, max=99, value=25, step=1, description='Age:')
    selected_gender = widgets.ToggleButtons(options = user_options_sorted_values(data_dict.sex),\
                                            value = "Male",\
                                            description='Sex:', \
                                            disabled=False, button_style='', )
    selected_percentile = widgets.IntSlider(min=0, max=100, step=10, value=50, description='Percentile:',)
    text_generation_button = Button(description="Generate Text")
    
#     display(selected_age, selected_gender, selected_percentile, output)
    
    selected_age_str= str(selected_age.value)

    selection_filter = data_dict[(data_dict.ages.str.contains(selected_age_str)) & 
                              (data_dict.sex.str.contains(selected_gender.value))]
#                              | (data_dict.sex == "Both"))]   
    
    with data_output:
            display(selection_filter)
    
    def selection_filtering(age_group, sex):
        output.clear_output()
        data_output.clear_output()

        selected_age_str= str(selected_age.value)
        
        selection_filter = data_dict[(data_dict.ages.str.contains(selected_age_str)) & 
                                  (data_dict.sex.str.contains(selected_gender.value))]
#                                    | (data_dict.sex == "Both"))]   
        with data_output:
            display(selection_filter)

    def selected_age_eventhandler(change):
        selection_filtering(change.new, selected_age.value)
    def selected_gender_eventhandler(change):
        selection_filtering(selected_gender.value, change.new)
    def selected_percentile_eventhandler(change):
        selection_filtering(selected_percentile.value, change.new)

    selected_age.observe(selected_age_eventhandler, names='value')
    selected_gender.observe(selected_gender_eventhandler, names='value')
    selected_percentile.observe(selected_percentile_eventhandler, names='value')

    list_of_variable_inputs = selection_filter["variables"].values[0:]
    variable_inputs = ', '.join(list_of_variable_inputs).replace(" ", "")
    variable_inputs = variable_inputs.split(',')
    
user_selection()

In [97]:
def get_demographics_for_selection():
# calculate percentile_input_per_variable
    global percentile_input, data

    data = pd.merge(acs_site_sum.loc[acs_site_sum['variables'].isin(variable_inputs)], \
                   selection_filter, how="outer", on="variables")    
    data.round({'sum_in_area': 0})
    data.sort_values("sum_in_area", axis = 0, ascending = True, inplace = True)

    percentile_input = selected_percentile.value / 100
    
# split these up into the diff bins for different types of variable groups 
    global language, education, school_enrollment, family_household_income, nonfamily_household_income, household_type

    for item,i in enumerate(data):       
        language = data[(data["variable_group"].str.contains('Language'))]
        education = data[(data["variable_group"].str.contains('Educational Attainment'))]
        school_enrollment = data[(data["variable_group"].str.contains('School'))]
        family_household_income = data[(data["variable_group"].str.contains('Family'))]
        nonfamily_household_income = data[(data["variable_group"].str.contains('Nonfamily'))]
        household_type = data[(data["variable_group"].str.contains('Households'))]
#       travel_time_to_work = data[(data["variable_group"].str.contains('Travel Time'))].sort_values(by='sum_in_area')
        # means_of_transportation = data[(data["variable_group"].str.contains('Means of Transportation'))].sort_values(by='sum_in_area')

#Calculate individual percentile values
        global sum_for_percentile_language,sum_for_percentile_education,sum_for_percentile_school_enrollment,sum_for_percentile_household_type,sum_for_percentile_family_household_income,sum_for_percentile_nonfamily_household_income, sum_for_household_type
        sum_for_percentile_language = language.sum_in_area.quantile(percentile_input).astype(int).astype(str)
        sum_for_percentile_education = education.sum_in_area.quantile(percentile_input).astype(int).astype(str)
        sum_for_percentile_school_enrollment = school_enrollment.sum_in_area.quantile(percentile_input).astype(int).astype(str)    
        sum_for_percentile_family_household_income = family_household_income.sum_in_area.quantile(percentile_input).astype(int).astype(str)
        sum_for_percentile_nonfamily_household_income = nonfamily_household_income.sum_in_area.quantile(percentile_input).astype(int).astype(str)
        sum_for_percentile_household_type = household_type.sum_in_area.quantile(percentile_input).astype(int).astype(str)
    
get_demographics_for_selection()

In [98]:
# df.sum_in_area = df.sum_in_area.dropna().apply(np.int32).round(decimals=0)
# df

In [99]:
def parse_tables_for_percentile_value():
#generating new transposed table with only the two fields needed : variables and sum in area.
#using other variables makes transposition weird
    global household_type_transposed, language_transposed, education_transposed, family_household_income_transposed, nonfamily_household_income_transposed,household_type_transposed, school_enrollment_transposed

    language_transposed = language.filter(["variables", "sum_in_area"]).T
    language_transposed.columns = language_transposed.iloc[0]
    language_transposed = language_transposed[1:]

    education_transposed = education.filter(["variables", "sum_in_area"]).T
    education_transposed.columns = education_transposed.iloc[0]
    education_transposed = education_transposed[1:]

    household_type_transposed = household_type.filter(["variables", "sum_in_area"]).T
    household_type_transposed.columns = household_type_transposed.iloc[0]
    household_type_transposed = household_type_transposed[1:]
    
    family_household_income_transposed = family_household_income.filter(["variables", "sum_in_area"]).T
    family_household_income_transposed.columns = family_household_income_transposed.iloc[0]
    family_household_income_transposed = family_household_income_transposed[1:]    
    
    nonfamily_household_income_transposed = nonfamily_household_income.filter(["variables", "sum_in_area"]).T
    nonfamily_household_income_transposed.columns = nonfamily_household_income_transposed.iloc[0]
    nonfamily_household_income_transposed = nonfamily_household_income_transposed[1:]  
    
    school_enrollment_transposed = school_enrollment.filter(["variables", "sum_in_area"]).T
    school_enrollment_transposed.columns = school_enrollment_transposed.iloc[0]
    school_enrollment_transposed = school_enrollment_transposed[1:]    

parse_tables_for_percentile_value()

In [125]:
# def get_range_for_each_variable():

transposed = [education_transposed, family_household_income_transposed, household_type_transposed, \
        language_transposed, nonfamily_household_income_transposed, school_enrollment_transposed]
data.sort_values(by=['variable_group', 'sum_in_area'], ascending=[True, True], inplace=True)
data_sorted = data.reset_index()

ranges=[]

for df in transposed:
    for item, i in enumerate(df.columns):
        if item == 0:
            first_range = np.arange(df.max()[item]).astype(int)
            ranges.append([first_range])
        else:
            other_ranges = np.arange(df.min()[item-1]+1, \
                                   df.max()[item]+1).astype(int)
            ranges.append([other_ranges])
            
        range_table = pd.DataFrame(data=ranges, index=None, columns=["range_per_variable"])
        range_table = range_table.reset_index(drop=True)
        
range_table_all = pd.merge(range_table, data_sorted, left_index=True, right_index=True, on=None)
range_table_all["range_per_variable"] = range_table_all["range_per_variable"].astype(str)

range_table_all.head()

,range_per_variable,index,variables,sum_in_area,sex,age_group,variable_group,variable_name,ages
0,[ 0 1 2 3 4 5 6 7 8 ...,26,S1501_C03_018E,1990.428048,Male,25 to 34 years,Educational Attainment,Bachelor's degree or higher,"25,26, 27, 28, 29, 30, 31, 32, 33, 34"
1,[1991 1992 1993 1994 1995 1996 1997 1998 1999 ...,25,S1501_C03_017E,2460.207098,Male,25 to 34 years,Educational Attainment,High school graduate or higher,"25,26, 27, 28, 29, 30, 31, 32, 33, 34"
2,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 ...,8,S1901_C01_005E,25.005896,Male Female Both,all ages,Family Household Income In The Past 12 Months ...,"Family Household Income $25,000 to $34,999","5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,..."
3,[26 27 28],9,S1901_C01_006E,27.991213,Male Female Both,all ages,Family Household Income In The Past 12 Months ...,"Family Household Income $35,000 to $49,999","5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,..."
4,[28 29 30 31],6,S1901_C01_003E,31.961707,Male Female Both,all ages,Family Household Income In The Past 12 Months ...,"Family Household Income $10,000 to $14,999","5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,..."


In [196]:
# for item,i in enumerate(range_table_all.index):
#     if range_table_all["variable_group"].str.contains('Educational').any():        
#         print("help")
# #         result = range_table_all[range_table_all['ranges'].str.contains(sum_for_percentile_education)]
# #         result_variable = result["variables"].values[0]
sum_for_percentile_education

'2225'

In [181]:
range_table_cumulative

,range_per_variable,index,variables,sum_in_area,sex,age_group,variable_group,variable_name,ages
0,[ 0 1 2 3 4 5 6 7 8 ...,26,S1501_C03_018E,1990.428048,Male,25 to 34 years,Educational Attainment,Bachelor's degree or higher,"25,26, 27, 28, 29, 30, 31, 32, 33, 34"
1,[1991 1992 1993 1994 1995 1996 1997 1998 1999 ...,25,S1501_C03_017E,2460.207098,Male,25 to 34 years,Educational Attainment,High school graduate or higher,"25,26, 27, 28, 29, 30, 31, 32, 33, 34"
2,[ 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 ...,8,S1901_C01_005E,25.005896,Male Female Both,all ages,Family Household Income In The Past 12 Months ...,"Family Household Income $25,000 to $34,999","5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,..."
3,[26 27 28],9,S1901_C01_006E,27.991213,Male Female Both,all ages,Family Household Income In The Past 12 Months ...,"Family Household Income $35,000 to $49,999","5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,..."
4,[28 29 30 31],6,S1901_C01_003E,31.961707,Male Female Both,all ages,Family Household Income In The Past 12 Months ...,"Family Household Income $10,000 to $14,999","5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,..."
5,[32],13,S1901_C01_010E,32.345257,Male Female Both,all ages,Family Household Income In The Past 12 Months ...,"Family Household Income $150,000 to $199,999","5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,..."
6,[33 34 35 36 37 38],7,S1901_C01_004E,37.457826,Male Female Both,all ages,Family Household Income In The Past 12 Months ...,"Family Household Income $15,000 to $24,999","5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,..."
7,[38 39 40],14,S1901_C01_011E,40.414763,Male Female Both,all ages,Family Household Income In The Past 12 Months ...,"Family Household Income $200,000 or more","5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,..."
8,[41 42 43 44 45 46 47],5,S1901_C01_002E,47.109273,Male Female Both,all ages,Family Household Income In The Past 12 Months ...,"Family Household Income Less than $10,000","5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,..."
9,[48],11,S1901_C01_008E,47.813793,Male Female Both,all ages,Family Household Income In The Past 12 Months ...,"Family Household Income $75,000 to $99,999","5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,..."


In [291]:
# def generate_info_for_text():
global descriptors
descriptors = []

for item,i in enumerate(range_table_cumulative.index):
    if sum_for_percentile_education.astype(int) > 0 :
        education_only = range_table_cumulative[(range_table_cumulative["variable_group"].str.contains('Educational'))]
        result = education_only[education_only["range_per_variable"].str.contains(sum_for_percentile_education)]
# #         result_variable = result["variable_name"].values[0]
        descriptors.append(result["variable_name"].values[0])
    
    if sum_for_percentile_language.astype(int) > 0 :
        language_only = range_table_cumulative[(range_table_cumulative["variable_group"].str.contains('Language'))]
        result = language_only[language_only["range_per_variable"].str.contains(sum_for_percentile_language)]
        descriptors.append(result["variable_name"].values[0])
    
    if sum_for_percentile_household_type.astype(int) > 0:
        household_type_only = range_table_cumulative[(range_table_cumulative["variable_group"].str.contains('Households'))]
        result = household_type_only[household_type_only["range_per_variable"].str.contains(sum_for_percentile_household_type)]
        descriptors.append(result["variable_name"].values[0])
    
    if sum_for_percentile_family_household_income.astype(int) > 0:
        family_household_income_only = range_table_cumulative[(range_table_cumulative["variable_group"].str.contains('Family'))]
        result = family_household_income_only[family_household_income_only["range_per_variable"].str.contains(sum_for_percentile_family_household_income)]
        descriptors.append(result["variable_name"].values[0])
    
    if sum_for_percentile_nonfamily_household_income.astype(int) > 0:
        nonfamily_household_income_only = range_table_cumulative[(range_table_cumulative["variable_group"].str.contains('Nonfamily'))]
        result = nonfamily_household_income_only[nonfamily_household_income_only["range_per_variable"].str.contains(sum_for_percentile_nonfamily_household_income)]
        descriptors.append(result["variable_name"].values[0])
        
    if sum_for_percentile_school_enrollment.astype(int) > 0:
        school_enrollment_only = range_table_cumulative[(range_table_cumulative["variable_group"].str.contains('School'))]
        result = school_enrollment_only[school_enrollment_only["range_per_variable"].str.contains(sum_for_percentile_school_enrollment)]
        descriptors.append(result["variable_name"].values[0])
    
descriptors = np.unique(descriptors)
descriptors
# generate_info_for_text()

array(['Family Household Income $15,000 to $24,999',
       'High school graduate or higher',
       'Nonfamily Household Income $200,000 or more',
       'Nonfamily households', 'Spanish'], dtype='<U43')

In [ ]:
# def generate_info_for_text():
# #     global range_table_cumulative_all
# #     range_table_cumulative_all = range_table_cumulative_all..fillna()
#     global descriptors, result, result_variable, result_text
#     descriptors=[]
    
#     for item,i in enumerate(range_table_cumulative.index_y):
#         if range_table_cumulative["sum_ranges"].str.contains(sum_for_percentile_language).any():        
#             result = range_table_cumulative[range_table_cumulative['sum_ranges'].str.contains(sum_for_percentile_language)]
#             result_variable = result["variables"].values[0]

#             #So returning the data dictionary for this 
#             dict_result = data_dict[data_dict["variables"].str.contains(result_variable)]
#             result_text = dict_result["variable_name"].values[0]
            
#             #then append to empty dataframe that will have the data for text generation
#             descriptors.append([result_text])
        
#         elif range_table_cumulative1["sum_ranges"].str.contains(sum_for_percentile_education).any():
#             result = range_table_cumulative1[range_table_cumulative1['sum_ranges'].str.contains(sum_for_percentile_education)]
#             result_variable = result["variables"].values[0]

#             #So returning the data dictionary for this 
#             dict_result = data_dict[data_dict["variables"].str.contains(result_variable)]
#             result_text = dict_result["variable_name"].values[0]
            
#             descriptors.append([result_text])

#         elif range_table_cumulative2["sum_ranges"].str.contains(sum_for_percentile_family_household_income).any():
#             result = range_table_cumulative2[range_table_cumulative2['sum_ranges'].str.contains(sum_for_percentile_family_household_income)]
#             result_variable = result["variables"].values[0]

#             #So returning the data dictionary for this 
#             dict_result = data_dict[data_dict["variables"].str.contains(result_variable)]
#             result_text = dict_result["variable_name"].values[0]
            
#             descriptors.append([result_text])

#         elif range_table_cumulative3["sum_ranges"].str.contains(sum_for_percentile_nonfamily_household_income).any():
#             result = range_table_cumulative3[range_table_cumulative3['sum_ranges'].str.contains(sum_for_percentile_nonfamily_household_income)]
#             result_variable = result["variables"].values[0]

#             #So returning the data dictionary for this 
#             dict_result = data_dict[data_dict["variables"].str.contains(result_variable)]
#             result_text = dict_result["variable_name"].values[0]
            
#             descriptors.append([result_text])
        
# generate_info_for_text()

In [ ]:
result,text

In [ ]:
text_generation_button = Button(description="Generate Text")

In [ ]:
def replace_text():
    global variable_text, subject_description, percentile_string, resident_string, gender_text

    percentile_string = "This description represents the top "+ str(selected_percentile.value) + "% of this area's residents."
    print(percentile_string)
    
    if (selected_gender.value == "Male"):
        gender_text = "she"
    elif (selected_gender.value == "Female"):
        gender_text = "he"
    elif (selected_gender.value == "Both"):
        gender_text = "they"
        
    if (selected_age.value <25 ) & (selected_gender.value == "Female"):
        subject_description = "As a young woman, you "
    elif (selected_age.value <25 ) & (selected_gender.value == "Male"):
        subject_description = "As a young man, you "
    
    if (result_text == "Speak a Language other than English at Home"):
        variable_text = "In addition to english, " + gender_text + " speaks " + + " at home."
    else:
        variable_text = "In addition to english, " + gender_text + " speaks " + + " at home."
#     This young woman is well-educated and has at least a bachelor’s degree. In addition to English, she speaks Spanish at home. She lives with her family, and her household is quite wealthy. She drives to work alone on her short commute. 

    resident_string = variable_text + subject_description + percentile_string + gender_text
    
    with output:
        display(resident_string)
    
replace_text()

In [ ]:
selected_gender.value

In [ ]:
# resident_string = "help"

#TEXT GENERATION
def text_generation(b):
    output.clear_output()
    data_output.clear_output()
    
    user_selection()
#     descriptor_generation()
#     range_per_variable_calculation()
#     generate_base_text()
    
    replace_text()    
    
text_generation_button.on_click(text_generation)

In [ ]:
def display_dashboard():
    user_selection()
    
    item_layout = widgets.Layout(margin='0 0 10px 0')
    
    with output:
        display(resident_string)
    with data_output:
        display(range_table_cumulative_all)
        
    input_widgets = widgets.VBox(
        [selected_age, selected_gender, selected_percentile, text_generation_button],
        layout=item_layout)
    
    tab = widgets.Tab([output, data_output],
        layout=item_layout)
    tab.set_title(0, 'Narrative')
    tab.set_title(1, 'Dataset Exploration')
    
    dashboard = widgets.VBox([input_widgets, tab])
    display(dashboard)

display_dashboard()

In [ ]:
# widget_control = WidgetControl(widget=text_generation_button, position='topright')
# m.add_control(widget_control)

# m